In [26]:
!pip install pandas pydantic langchain langchain_community langchain_openai langchain_anthropic langchain_google_genai deepeval rank_bm25 faiss-cpu duckduckgo-search

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 10.4 MB/s eta 0:00:00 0:00:01


In [8]:
import os
import re
import json
from typing import List, Tuple, Optional

import pandas as pd
from pydantic import BaseModel, Field
from langchain.docstore.document import Document
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.tools import DuckDuckGoSearchResults
from dotenv import load_dotenv

from deepeval import evaluate
from deepeval.metrics import GEval, FaithfulnessMetric, ContextualRelevancyMetric
from deepeval.test_case import LLMTestCase, LLMTestCaseParams



/home/aragy/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
load_dotenv("/home/aragy/HumaAI/Huma.ai-assessment/Section_2_to_6/.env")

True

# Section 2 : Setting Up the Retrieval System

The retrieval system is based in two techniques working together:
- Reciprocal Rank Fusion (https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf):
    using FAISS as vectorstore to perform a similarity search based in vector representation, as BM25 to seach similarily based on statistics

- CRAG(Corrective RAG)
    Extends the standard RAG approach by dynamically evaluating and correcting the retrieval process, combining the power of vector databases(in this case RRF retrieval), web search, and language models to provide accurate and context-aware responses to user queries. 

In [10]:
class DocumentRetriever:
    def __init__(self, bm25_docs: List[str], faiss_docs: List[str]):
        self.bm25_retriever = BM25Retriever.from_texts(
            bm25_docs, metadatas=[{"source": 1}] * len(bm25_docs)
        )
        self.bm25_retriever.k = 5        
        embedding = OpenAIEmbeddings()
        faiss_vectorstore = FAISS.from_texts(
            faiss_docs, embedding, metadatas=[{"source": 2}] * len(faiss_docs)
        )
        self.faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 5})

        self.ensemble_retriever = EnsembleRetriever(
            retrievers=[self.bm25_retriever, self.faiss_retriever], weights=[0.5, 0.5]
        )

    def retrieve(self, query: str) -> List[Document]:
        return self.ensemble_retriever.invoke(query)

In [11]:
class RetrievalEvaluatorInput(BaseModel):
    relevance_score: float = Field(..., description="The relevance score of the document to the query. the score should be between 0 and 1.")

In [12]:
class Evaluator:
    def __init__(self, llm):
        self.llm = llm

    def evaluate_relevance(self, query: str, document: str) -> float:
        prompt = PromptTemplate(
            input_variables=["query", "document"],
            template="On a scale from 0 to 1, how relevant is the following document to the query? Query: {query}\nDocument: {document}\nRelevance score:"
        )
        chain = prompt | self.llm.with_structured_output(RetrievalEvaluatorInput)
        input_variables = {"query": query, "document": document}
        result = chain.invoke(input_variables).relevance_score
        return result

In [13]:
class KnowledgeRefinementInput(BaseModel):
    key_points: str = Field(..., description="The document to extract key information from.")

In [14]:

class KnowledgeRefiner:
    def __init__(self, llm):
        self.llm = llm

    def refine(self, document: str) -> List[str]:
        prompt = PromptTemplate(
            input_variables=["document"],
            template="Extract the key information from the following document in bullet points:\n{document}\nKey points:"
        )
        chain = prompt | self.llm.with_structured_output(KnowledgeRefinementInput)
        input_variables = {"document": document}
        result = chain.invoke(input_variables).key_points
        return [point.strip() for point in result.split('\n') if point.strip()]

In [15]:
class QueryRewriterInput(BaseModel):
    query: str = Field(..., description="The query to rewrite.")

In [16]:
class QueryRewriter:
    def __init__(self, llm):
        self.llm = llm

    def rewrite(self, query: str) -> str:
        prompt = PromptTemplate(
            input_variables=["query"],
            template="Rewrite the following query to make it more suitable for a web search:\n{query}\nRewritten query:"
        )
        chain = prompt | self.llm.with_structured_output(QueryRewriterInput)
        input_variables = {"query": query}
        return chain.invoke(input_variables).query.strip()

In [17]:
class WebSearcher:
    def __init__(self, llm):
        self.llm = llm
        self.search = DuckDuckGoSearchResults()

    def search_and_refine(self, query: str) -> Tuple[List[str], List[Tuple[str, str]]]:
        rewritten_query = QueryRewriter(self.llm).rewrite(query)
        web_results = self.search.run(rewritten_query)
        web_knowledge = KnowledgeRefiner(self.llm).refine(web_results)
        sources = self.parse_search_results(web_results)
        return web_knowledge, sources

    @staticmethod
    def parse_search_results(results_string: str) -> List[Tuple[str, str]]:
        try:
            pattern = r'snippet: (.*?), title: (.*?), link: (https?://[^\s,]+)'
            matches = re.findall(pattern, results_string)
            data = [{"snippet": snippet, "title": title, "link": link} for snippet, title, link in matches]
            results = json.loads(json.dumps(data, indent=4))
            return [(result.get('title', 'Untitled'), result.get('link', '')) for result in results]
        except json.JSONDecodeError:
            print("Error parsing search results. Returning empty list.")
            return []

In [18]:
class ResponseGenerator:
    def __init__(self, llm):
        self.llm = llm

    def generate(self, query: str, knowledge: str, sources: List[Tuple[str, str]]) -> str:
        response_prompt = PromptTemplate(
            input_variables=["query", "knowledge", "sources"],
            template="Based on the following knowledge, answer the query. Include the sources with their links (if available) at the end of your answer:\nQuery: {query}\nKnowledge: {knowledge}\nSources: {sources}\nAnswer:"
        )
        input_variables = {
            "query": query,
            "knowledge": knowledge,
            "sources": "\n".join([f"{title}: {link}" if link else title for title, link in sources])
        }
        response_chain = response_prompt | self.llm
        return response_chain.invoke(input_variables).content

In [19]:
class QueryProcessor:
    def __init__(self, retriever: DocumentRetriever, evaluator: Evaluator, web_searcher: WebSearcher, llm):
        self.retriever = retriever
        self.evaluator = evaluator
        self.web_searcher = web_searcher
        self.llm = llm

    def process(self, query: str, eval_documents: bool = True) -> str:
        """
        Process the query and generate a response.

        Args:
            query (str): The query string.
            eval_documents (bool): Whether to calculate evaluation scores or just use the retrieved documents.

        Returns:
            str: The generated response.
        """
        retrieved_docs = self.retriever.retrieve(query)
        
        if eval_documents:
            eval_scores = [self.evaluator.evaluate_relevance(query, doc.page_content) for doc in retrieved_docs]
            max_score = max(eval_scores)

            if max_score > 0.7:
                best_doc = retrieved_docs[eval_scores.index(max_score)]
                final_knowledge = best_doc.page_content
                sources = [("Retrieved document", "")]
            elif max_score < 0.3:
                final_knowledge, sources = self.web_searcher.search_and_refine(query)
            else:
                best_doc = retrieved_docs[eval_scores.index(max_score)]
                retrieved_knowledge = KnowledgeRefiner(self.llm).refine(best_doc.page_content)
                web_knowledge, web_sources = self.web_searcher.search_and_refine(query)
                final_knowledge = "\n".join(retrieved_knowledge + web_knowledge)
                sources = [("Retrieved document", "")] + web_sources
        else:
            final_knowledge = retrieved_docs[0].page_content
            sources = [("Retrieved document", "")]

        response = ResponseGenerator(self.llm).generate(query, final_knowledge, sources)
        return response

In [24]:
#df = pd.read_csv('/home/aragy/Huma.ai-assessment/Dataset/cleaned_dataset.csv')
df = pd.read_csv('/home/aragy/HumaAI/Huma.ai-assessment/Dataset/cleaned_dataset.csv')
documents = [row['question'] + ' ' + row['answer_solution'] for _, row in df.iterrows()]

retriever = DocumentRetriever(documents, documents)


In [16]:
retriever.retrieve("What are the Keytruda's side effects?")

[Document(metadata={'source': 1}, page_content='What are the common side effects of Keytruda? Common side effects include fatigue, nausea, and skin rash.'),
 Document(metadata={'source': 1}, page_content='Can Keytruda cause immune-related adverse effects? Yes, Keytruda can cause immune-related adverse effects such as colitis, hepatitis, and pneumonitis.'),
 Document(metadata={'source': 1}, page_content='What were the side effects noted in the KEYNOTE-006 trial? Common side effects included fatigue, itching, and diarrhea.'),
 Document(metadata={'source': 2}, page_content='Are there specific side effects of Keytruda that NSCLC patients should monitor? NSCLC patients should monitor for cough, shortness of breath, and chest pain, as these could indicate immune-related pneumonitis.'),
 Document(metadata={'source': 2}, page_content='What should patients report immediately while on Keytruda treatment? Patients should report any new or worsening symptoms such as cough, chest pain, or changes i

In [27]:
llm = ChatOpenAI(model="gpt-4o-mini", max_tokens=1000, temperature=0)
evaluator = Evaluator(llm)
web_searcher = WebSearcher(llm)

### Section 6 - Expanding the RAG System to the Internet

In [28]:
web_searcher.search("What are the Keytruda's side effects?")

/tmp/ipykernel_2986795/96624778.py:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  web_searcher.search("What are the Keytruda's side effects?")


"snippet: Keytruda is a biologic that belongs to the programmed cell death protein 1 (PD-1) inhibitor drug class. Read on to learn about potential common, mild, and serious side effects of Keytruda. For a ..., title: Keytruda side effects: What they are and how to manage them, link: https://www.medicalnewstoday.com/articles/drugs-keytruda-side-effects, snippet: These side effects can vary depending on whether Keytruda is used alone or with other cancer drugs. Examples of Keytruda's commonly reported side effects include: diarrhea. fatigue. loss of ..., title: Keytruda Side Effects: Common to Serious, Explained - Healthline, link: https://www.healthline.com/health/drugs/keytruda-side-effects, snippet: Check with your doctor or nurse immediately if any of the following side effects occur while taking pembrolizumab: More common. black, tarry stools. bladder pain. bloating or swelling of the face, arms, hands, lower legs, or feet. bloody or cloudy urine. blurred vision. body aches or pain.

In [29]:
web_searcher.search_and_refine("What are the Keytruda's side effects?")

(['- Keytruda is a biologic drug that belongs to the PD-1 inhibitor class.',
  '- Side effects can vary based on whether Keytruda is used alone or with other cancer drugs.',
  '- Commonly reported side effects include:',
  '- Diarrhea',
  '- Fatigue',
  '- Loss of appetite',
  '- Pain (stomach, muscles, bones, joints)',
  '- Rash or itching',
  '- Fever',
  '- Cough',
  '- Serious side effects to watch for include:',
  '- Black, tarry stools',
  '- Bladder pain',
  '- Bloating or swelling of face, arms, hands, lower legs, or feet',
  '- Bloody or cloudy urine',
  '- Blurred vision',
  '- Body aches or pain',
  '- It is important to consult a doctor if serious side effects occur.'],
 [('Keytruda side effects: What they are and how to manage them',
   'https://www.medicalnewstoday.com/articles/drugs-keytruda-side-effects'),
  ('Keytruda Side Effects: Common to Serious, Explained - Healthline',
   'https://www.healthline.com/health/drugs/keytruda-side-effects'),
  ('Keytruda Side Effects:

# Section 3: Integrating the Generation Component





In [30]:
llm = ChatOpenAI(model="gpt-4o-mini", max_tokens=1000, temperature=0)

In [31]:

evaluator = Evaluator(llm)
web_searcher = WebSearcher(llm)
processor_gpt = QueryProcessor(retriever, evaluator, web_searcher, llm)

In [21]:
# response = processor_gpt.process("What are the Keytruda's side effects?")
# print(response)

Keytruda (pembrolizumab) is an immunotherapy drug used to treat various types of cancer. Common side effects of Keytruda include:

- Fatigue
- Nausea
- Skin rash

These side effects can vary in intensity and may not occur in every patient. It's important for patients to discuss any side effects they experience with their healthcare provider.

Sources: Retrieved document.


In [22]:
# response = processor_gpt.process("What is the indication for using Keytruda?")
# print(response)

Keytruda (pembrolizumab) is an immunotherapy medication primarily indicated for the treatment of various types of cancer, including:

1. **Non-small cell lung cancer (NSCLC)** - It is particularly effective in patients with PD-L1 expression, significantly improving survival rates.
2. **Melanoma**
3. **Head and neck squamous cell cancer (HNSCC)**
4. **Classical Hodgkin Lymphoma**
5. **Endometrial carcinoma**
6. **Triple-negative breast cancer**

Keytruda functions as a checkpoint inhibitor, enhancing the immune system's ability to fight cancer. It is intended for long-term treatment, with recommended dosages of 200 mg once every 3 weeks or 400 mg once every 6 weeks. Treatment effectiveness is monitored through imaging, blood tests, and laboratory tests.

For more detailed information, you can refer to the following sources:
- [Drugs.com - Keytruda: Uses, Dosage, Side Effects, Warnings](https://www.drugs.com/keytruda.html)
- [Medical News Today - Keytruda (pembrolizumab): Side effects, u

In [23]:
# response = processor_gpt.process("How Keytruda perform in cancer remission?")
# print(response)

Keytruda (pembrolizumab) has shown significant effectiveness in promoting cancer remission, particularly in specific types of cancer such as non-small cell lung cancer (NSCLC) and muscle-invasive urothelial carcinoma (MIUC). 

1. **Non-Small Cell Lung Cancer (NSCLC)**: Keytruda is particularly effective in patients with high PD-L1 expression, leading to improved survival rates. The use of PD-1 immune checkpoint inhibitors like Keytruda has been associated with prolonged overall survival (OS) in various cancer patients.

2. **Muscle-Invasive Urothelial Carcinoma (MIUC)**: In patients with high-risk MIUC, adjuvant treatment with Keytruda has resulted in improved disease-free survival (DFS) compared to observation alone after surgery. This suggests that Keytruda can significantly enhance remission rates in this patient population.

3. **General Effectiveness**: The effectiveness of Keytruda can vary based on the type of cancer, its stage, and the patient's previous treatments. It has been

### Claude 3.5 sonnet

In [32]:
llm_claude = ChatAnthropic(temperature=0, model_name='claude-3-5-sonnet-20240620')

In [33]:
evaluator_claude = Evaluator(llm_claude)
web_searcher_claude = WebSearcher(llm_claude)
processor_claude = QueryProcessor(retriever, evaluator_claude, web_searcher_claude, llm_claude)

In [58]:
# response = processor_claude.process("What are the Keytruda's side effects?")
# print(response)

Based on the provided knowledge, the common side effects of Keytruda include:

1. Fatigue
2. Nausea
3. Skin rash

It's important to note that these are just the common side effects, and individual experiences may vary. Patients should always consult their healthcare provider for a comprehensive list of potential side effects and to discuss any concerns they may have about their treatment.

Source:
Retrieved document (no link available)


In [59]:
# response = processor_claude.process("What is the indication for using Keytruda?")
# print(response)

Based on the provided knowledge, Keytruda (pembrolizumab) is indicated for the treatment of various types of cancer, including:

1. Non-small cell lung cancer, particularly in patients with PD-L1 expression
2. Melanoma (skin cancer)
3. Adenocarcinomas
4. Endometrial carcinoma
5. Triple-negative breast cancer

Keytruda is an immunotherapy treatment classified as a checkpoint inhibitor. It works by helping the immune system fight cancer cells. The treatment has shown significant improvement in survival rates, especially for non-small cell lung cancer patients.

It's important to note that Keytruda is intended for long-term treatment, and its use is continuously being evaluated in numerous clinical trials for various cancers. Since its initial approval in September 2014, at least 20 new indications have been approved, and ongoing investigational studies are exploring new uses for pembrolizumab.

Sources:
1. Keytruda: Uses, Dosage, Side Effects, Warnings - Drugs.com: https://www.drugs.com/

In [60]:
# response = processor_claude.process("How Keytruda perform in cancer remission?")
# print(response)

Based on the provided knowledge, I cannot directly answer how Keytruda performs in cancer remission. The information given is specifically about Keytruda's effectiveness in treating non-small cell lung cancer, not about its performance in cancer remission.

However, I can share what is provided:

Keytruda has demonstrated significant improvement in survival rates for patients with non-small cell lung cancer who have PD-L1 expression. This suggests that Keytruda is effective in treating this specific type of cancer, but it does not provide information about cancer remission in general or for other types of cancer.

To accurately answer the query about Keytruda's performance in cancer remission, more specific and relevant information would be needed.

Source:
Retrieved document (no link available)


### Gemini

In [34]:
llm_gemini = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0,)

In [35]:
evaluator_gemini = Evaluator(llm_gemini)
web_searcher_gemini = WebSearcher(llm_gemini)
processor_gemini = QueryProcessor(retriever, evaluator_gemini, web_searcher_gemini, llm_gemini)

In [40]:
response = processor_gemini.process("What are the Keytruda's side effects?")
print(response)

Keytruda's common side effects include fatigue, nausea, and skin rash. 

**Source:** Retrieved document 



In [42]:
response = processor_gemini.process("What is the indication for using Keytruda?")
print(response)

Keytruda is indicated for the treatment of relapsed or refractory classical Hodgkin lymphoma after two or more lines of therapy. 

**Source:** Retrieved document 



In [43]:
response = processor_gemini.process("How Keytruda perform in cancer remission?")
print(response)

Keytruda demonstrated superior efficacy compared to chemotherapy in NSCLC patients in the KEYNOTE-456 trial. This superiority was observed in terms of overall survival and progression-free survival. 

**Source:** Retrieved document 



# Section 4: Evaluation and Optimization

### Test Preparation

In [36]:
class TestScenario:
    def __init__(self, question: str, gt_answer: str, retrieval_context: List[str], ):
        self.question = question
        self.gt_answer = gt_answer
        self.retrieval_context = retrieval_context        
        self.pred_answer = None

    def run(self, processor:QueryProcessor):

        self.pred_answer = processor.process(self.question)

        test_case_combined = LLMTestCase(
            input=self.question,
            expected_output=self.gt_answer,
            actual_output=self.pred_answer,
            retrieval_context=self.retrieval_context,
        )
        return test_case_combined

In [37]:
class MetricsEvaluator:
    def __init__(self,model_name:str) -> None:
        self.model_name = model_name
        self.correctness_metric = GEval(
            name="Correctness",
            model=model_name,
            evaluation_params=[
                LLMTestCaseParams.EXPECTED_OUTPUT,
                LLMTestCaseParams.ACTUAL_OUTPUT],
            evaluation_steps=[
                "Determine whether the actual output is factually correct based on the expected output."
            ],
        )
        self.faithfulness_metric = FaithfulnessMetric(
            threshold=0.7,
            model=model_name,
            include_reason=True
        )
        self.relevance_metric = ContextualRelevancyMetric(
            threshold=1,
            model=model_name,
            include_reason=True
        )


In [38]:
class TestRunner:
    def __init__(self, metrics_evaluator:MetricsEvaluator):
        self.metrics_evaluator = metrics_evaluator

    def run_test(self, test_case: LLMTestCase):

        self.metrics_evaluator.correctness_metric.measure(test_case)
        self.metrics_evaluator.faithfulness_metric.measure(test_case)
        self.metrics_evaluator.relevance_metric.measure(test_case)

        print(f"Correctness Score: {self.metrics_evaluator.correctness_metric.score}")
        print(f"Faithfulness Score: {self.metrics_evaluator.faithfulness_metric.score}")
        print(f"Contextual Relevancy Score: {self.metrics_evaluator.relevance_metric.score}")

### Scenarios

1. Keytruda efficacy and drug interactions

    * Question: "Are there any known interactions between Keytruda and other medications?"
    * Ground Truth : "Yes, Keytruda can interact with steroids and certain immunosuppressants, potentially affecting its efficacy and safety."
    * Context: Includes documents on Keytruda side effects, effectiveness, and interactions.

In [39]:
question = "Are there any known interactions between Keytruda and other medications?"
gt_answer = "Yes, Keytruda can interact with steroids and certain immunosuppressants, potentially affecting its efficacy and safety."

retrieval_context = [
    "Keytruda has interactions with steroids",
    "Keytruda is commonly used with other immunosuppressants"
]

scenario_1 = TestScenario(question=question, gt_answer=gt_answer, retrieval_context=retrieval_context)


2. Five-Year Survival Rate with Keytruda

    * Question: "What is the five-year survival rate for patients using Keytruda for lung cancer?"
    * Ground Truth: "No detailed information available on the given topic."
    * Context: Documents related to survival rates, but lacking detailed data for five-year outcomes.


In [40]:
question = "What is the five-year survival rate for patients using Keytruda for lung cancer?"
gt_answer = "No detailed information available on the given topic."

retrieval_context = [
    "No detailed information available on the given topic.",
    "Keytruda has improved survival rates, but specific five-year data may not be available."
]

scenario_2 = TestScenario(question=question, gt_answer=gt_answer, retrieval_context=retrieval_context)


3. Common Side Effects of Keytruda

    * Question: "What are the common side effects of Keytruda?"
    * Ground Truth: "Common side effects include fatigue, nausea, and skin rash."
    * Context: Medical literature on Keytruda's side effects.

In [41]:
question = "What are the common side effects of Keytruda?"
gt_answer = "Common side effects include fatigue, nausea, and skin rash."

retrieval_context = [
    "Common side effects include fatigue, nausea, and skin rash.",
    "Patients should report any severe or unexpected side effects to their healthcare provider."
]

scenario_3 = TestScenario(question=question, gt_answer=gt_answer, retrieval_context=retrieval_context)


4. Efficacy of Keytruda in NSCLC Patients

    * Question: "How effective is Keytruda in treating non-small cell lung cancer (NSCLC)?"
    * Ground Truth: "Keytruda has shown to improve survival rates significantly in non-small cell lung cancer patients with PD-L1 expression."
    * Context: Studies on the effectiveness of Keytruda in NSCLC treatment.

In [42]:
question = "How effective is Keytruda in treating non-small cell lung cancer (NSCLC)?"
gt_answer = "Keytruda has shown to improve survival rates significantly in non-small cell lung cancer patients with PD-L1 expression."

retrieval_context = [
    "Keytruda has shown to improve survival rates significantly in non-small cell lung cancer patients with PD-L1 expression.",
    "Some patients may see effects as early as 2 to 3 months into the treatment."
]

scenario_4 = TestScenario(question=question, gt_answer=gt_answer, retrieval_context=retrieval_context)


5. Response Rate in Melanoma Patients Treated with Keytruda

    * Question: "What is the response rate of melanoma patients treated with Keytruda?"
    * Ground Truth: "Approximately 40% of melanoma patients respond positively to Keytruda treatment."
    * Context: Documents on Keytruda's efficacy in melanoma treatment.

In [43]:
question = "What is the response rate of melanoma patients treated with Keytruda?"
gt_answer = "Approximately 40% of melanoma patients respond positively to Keytruda treatment."

retrieval_context = [
    "Approximately 40% of melanoma patients respond positively to Keytruda treatment.",
    "Response rates may vary based on individual patient factors."
]

scenario_5 = TestScenario(question=question, gt_answer=gt_answer, retrieval_context=retrieval_context)


### Run Test

In [44]:
model_name_gpt = "gpt-4o-mini"

In [45]:
test_scenarios = [scenario_1, scenario_2, scenario_3, scenario_4, scenario_5]

In [46]:
model_evaluator= MetricsEvaluator(model_name=model_name_gpt)

In [47]:
test_runner = TestRunner(model_evaluator)

### Testing GPT

In [ ]:
print("Testing processor GPT-4o-mini" )
for number, scenario in enumerate(test_scenarios):
    counter = number+1
    print("Scenario "+ str(counter))
    print("Question:"+ scenario.question)
    print("Ground Thruth:"+scenario.gt_answer)


    test_case = scenario.run(processor=processor_gpt)
    test_runner.run_test(test_case)
    print("------------------------------------")
    print("\n")

### Testing Gemini

In [ ]:
print("Testing processor Gemini" )
for number, scenario in enumerate(test_scenarios):
    counter = number+1
    print("Scenario "+ str(counter))
    print("Question:"+ scenario.question)
    print("Ground Thruth:"+scenario.gt_answer)


    test_case = scenario.run(processor=processor_gemini)
    test_runner.run_test(test_case)
    print("------------------------------------")
    print("\n")

### Testing Claude

In [49]:
print("Testing processor Claude" )
for number, scenario in enumerate(test_scenarios):
    counter = number+1
    print("Scenario "+ str(counter))
    print("Question:"+ scenario.question)
    print("Ground Thruth:"+scenario.gt_answer)


    test_case = scenario.run(processor=processor_claude)
    test_runner.run_test(test_case)
    print("------------------------------------")
    print("\n")

Testing processor Claude
Scenario 1
Question:Are there any known interactions between Keytruda and other medications?
Ground Thruth:Yes, Keytruda can interact with steroids and certain immunosuppressants, potentially affecting its efficacy and safety.


Output()